This notebook contains data prep and analysis for the wikipedia page graph.

Page graph is a directed graph where each node represents a page and each edge represents a hyperlink between two pages.

In [2]:
import pandas as pd
import time, json, os, random, traceback, pyperclip, importlib, queue
import matplotlib.pyplot as plt
import seaborn as sns
import src.wiki_analysis_utils as wiki_analysis_utils

_ = importlib.reload(wiki_analysis_utils)

In [3]:
data_root_dir = r'C:\Users\mohitvyas\MyDesktop\WikipediaDataset\data\\'

Prepare a small dataset for experimentation before jumping into the deep end.

1. Start with all categories tagged on: https://en.wikipedia.org/wiki/Coriander
2. ~~Do a BFS on undirected version of category graph starting from the above categories to obtain the category set of interest.~~ (This approach results in ~80% of all categories which is unhelpful. Intuition is to obtian small subgraph that is very similar to the seed page. Something like "set of all wiki articles about plant species like coriander")
    - ~~Alternative approach-1: do BFS on parent directed graph to obtain all ancenstors. Then do BFS on child directed graph to obtain all descendants of these ancestors.~~ (This also doesn't work)
    - Alternative approach-2: same as approach 1, except, keep only ancestors with <= N descendants. So total 2 constraints on anscenstors: (i) has <= N descendants (ii) contains at least 1 seed category in its descendants.
3. Get all articles that has at least 1 category from the category set of interest.
4. Keep only edges that are between articles in the above set to obtain the final dataset.

In [4]:
target_article_name = 'Coriander'
target_dir = data_root_dir + 'PageSubGraphs\\' + \
    wiki_analysis_utils.normalized_page_name(target_article_name).replace(' ', '_') + '\\'
os.makedirs(target_dir, exist_ok=True)

In [5]:
page_name_to_page_id, page_id_to_page_name, error_counts = \
    wiki_analysis_utils.load_page_name_to_id_map(data_root_dir, silent=True)

In [6]:
# load category name to id mappings from the category pages
categories, failure_counts = wiki_analysis_utils\
    .load_category_name_to_id_map(data_root_dir, silent=True)

In [7]:
target_page_ids = set([page_name_to_page_id[wiki_analysis_utils.normalized_page_name(target_article_name)]])
seed_categories = []
silent = True
start_time = time.time()
for partition in range(10):
    with open(data_root_dir + f'category_pages/part-{partition}.txt', 'r') as f:
        for line in f:
            if line=='': continue
            data = json.loads(line)
            if 'category_id' in data:
                for _, page_id in data['articles']:
                    if page_id in target_page_ids:
                        seed_categories.append(data['category_id'])
                        break
    if not silent:
        print(f"Processed till part {partition} in {(time.time() - start_time) / 60} minutes")

In [8]:
seed_category_names = [categories['id_to_name'][cat_id] for cat_id in seed_categories]
print (f"Found {len(seed_categories)} seed categories")
print (json.dumps(seed_category_names, indent=4))

Found 8 seed categories
[
    "plants described in 1753",
    "spices",
    "indian spices",
    "edible apiaceae",
    "herbs",
    "medicinal plants",
    "plants used in native american cuisine",
    "apioideae"
]


In [9]:
# load directed graphs for parent-child category relationships
parent_graph_adj_lists = {}
child_graph_adj_lists = {}
n_edges_loaded = 0
start_time = time.time()
silent=False
for _, row in pd.read_csv(data_root_dir + 'category_id_to_parent_category_ids.tsv', sep='\t').iterrows():
    cid1 = row['CategoryId']
    cid2 = row['ParentCategoryId']
    if cid1 not in parent_graph_adj_lists:
        parent_graph_adj_lists[cid1] = []
    parent_graph_adj_lists[cid1].append(cid2)
    if cid2 not in child_graph_adj_lists:
        child_graph_adj_lists[cid2] = []
    child_graph_adj_lists[cid2].append(cid1)
    n_edges_loaded += 1
    if n_edges_loaded % 500000 == 0 and not silent:
        print(f"Loaded {n_edges_loaded} edges in {(time.time() - start_time) / 60} minutes")

Loaded 500000 edges in 0.8095912933349609 minutes
Loaded 1000000 edges in 1.434980277220408 minutes
Loaded 1500000 edges in 2.0305140217145285 minutes
Loaded 2000000 edges in 2.5617505431175234 minutes
Loaded 2500000 edges in 3.1015053470929463 minutes
Loaded 3000000 edges in 3.608826283613841 minutes
Loaded 3500000 edges in 4.113653659820557 minutes
Loaded 4000000 edges in 4.683315519491831 minutes
Loaded 4500000 edges in 5.220084122816721 minutes


In [ ]:
# verify that the graph is a DAG


In [10]:
# BFS on the parent graph to find all the ancestors of the seed categories
print (f"Starting with {len(seed_categories)} seed categories")
ancestors = set(seed_categories)
nodes_to_visit = queue.Queue()
for seed_category in seed_categories:
    nodes_to_visit.put(seed_category)
while not nodes_to_visit.empty():
    node = nodes_to_visit.get()
    if node in parent_graph_adj_lists:
        for parent in parent_graph_adj_lists[node]:
            if parent not in ancestors:
                ancestors.add(parent)
                nodes_to_visit.put(parent)
print(f"Found {len(ancestors)} ancestor categories")

Starting with 8 seed categories
Found 18895 ancestor categories


In [14]:
# count number of descendants of each ancestor
# first identify root ancestors (i.e. ones with no parents in the ancestor set)
root_ancestors = set([ancestor for ancestor in ancestors if ancestor not in parent_graph_adj_lists])
print(f"Found {len(root_ancestors)} root ancestors")

node_to_descendant_count = {}
node_to_seed_category_descendant_count = {}

# recursively count descendants of each ancestor
# simulate recursion with a stack to avoid stack overflow
call_stack = [[root, 0] for root in root_ancestors]
while len(call_stack) > 0:
    node, idx = call_stack[-1]
    if node in child_graph_adj_lists:
        if idx < len(child_graph_adj_lists[node]):
            child = child_graph_adj_lists[node][idx]
            if child not in node_to_descendant_count:
                call_stack.append([child, 0])
            else:
                call_stack[-1][1] += 1
        else:
            count = 1
            for child in child_graph_adj_lists[node]:
                count += node_to_descendant_count[child]
            node_to_descendant_count[node] = count
            call_stack.pop()
    else:
        node_to_descendant_count[node] = 1
        call_stack.pop()

Found 222 root ancestors


KeyboardInterrupt: 

In [17]:
len(call_stack)
# len(ancestor_to_descendant_count)

42726194